**주피터 노트북 사용자:**
* 노트북에서 실행하실 분들은 `argparse` 삭제하시고 `batch_size`, `lr` 부분을 직접 함수에 인자로 넣으시면 되겠습니다.
* 코드 중간 `if __name__ == '__main__':` 은 파이썬에서 C/C++의 메인함수에 해당하는 부분입니다. 노트북에서는 다르게 작동하실 껍니다. 따라서 해당 라인을 삭제하고 함수만 실행시키시면 될겁니다.
* 저는 Multi-GPU를 사용하기 때문에 Multi-GPU가 불필요하신 분들은 해당 라인을 삭제하시고, Automatic Mixed Precision은 파이토치 1.5 버전부터 네이티브로 만들어진 기능입니다. 1.5 이하를 사용하시는 분들은 파이토치를 버전업하거나 해당 라인을 수정해서 사용하시기 바랍니다. 


**추가적인 팁:**
* 앙상블, 이미지 어그멘테이션, 스케줄러, 모형 변경 및 커스텀, 커스텀 로스 펑션 등을 순서대로 적용해보면서 자기만의 베이스라인과의 성능 차이를 기록하고 비교해보시기 바랍니다.
* `seed`를 고정하지 않았습니다. 병렬처리, 개발환경 등 모든 부분에 따라 다르기 때문에 캐글과 같이 비슷한 환경을 제공하거나 데이터셋이 너무 구려 가중치 초기화를 `seed`로 극복할 경우가 아닌 이상, 잘 학습된 모형의 경우 오차가 그리 크지 않을 겁니다.
* **중요: 모형을 커스텀할 때 꼭 레이어 초기화 방법을 설정해주시기 바랍니다. 성능 차이가 매우 큽니다.**
* **중요: 사용하실 모형과 딥러닝 프레임워크에 따라 인풋 데이터 형식이 다릅니다. 이미지 경우 채널 순서부터 정규화 방법까지 성능 차이가 매우 크므로 공식 문서를 꼭 읽고 따르시길 바랍니다. `torchvision.models`는 아래에 설명해놨습니다.**
* **중요: 코드공유에 공유된 Dacon.Jin님의 TTA를 적용해보시기 바랍니다. 성능 차이가 매우 큽니다.**

**참고: 코딩 스타일은 `<PEP8>`을 지킵니다. 파이썬 인덴트는 공백 4칸인데, 데이콘에 노트북 업로드 시 공백이 2칸으로 변경되네요. 4칸으로 수정해서 사용하시기 바랍니다.**

In [ ]:
import os
import argparse
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import KFold

import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torchvision import transforms
from torchvision.models import resnet101

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', default=512, type=int)
parser.add_argument('--lr', default=1e-3, type=float)
args = parser.parse_args()

## 1. Out-of-Fold 전략을 위한 함수 정의

In [ ]:
def split_dataset(path: os.PathLike) -> None:
    df = pd.read_csv(path)
    kfold = KFold(n_splits=5)
    for fold, (train, valid) in enumerate(kfold.split(df, df.index)):
        df.loc[valid, 'kfold'] = int(fold)

    df.to_csv('data/split_kfold.csv', index=False)

## 2. 커스텀 데이터셋 정의 

In [ ]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

## 3. 이미지 어그멘테이션 정의

`torchvision.models`의 모형을 이용하여 transfer learning을 적용하고 싶으실 때 파이토치 공식 지원 `pillow`를 사용하지 않고, `opencv`를 사용하실려면:
* 채널 순서는 RGB
* `uint8` 0~255 값을 [0, 1] 값으로 정규화
* 정규화된 값을 ImageNet의 mean, std로 다시 정규화

어그멘테이션 적용 시, 해당 데이터셋의 특징을 잘 파악하고 적용하셔야 합니다. `albumentations`은 `ndarray`로 받습니다. `pillow`를 넘파이로 변경하시기 바랍니다.
본 대회의 데이터셋은 1채널이라 3채널로 변경해서 사용하시면 되겠습니다. 

In [ ]:
transforms_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

## 4. 모형 정의 

In [ ]:
class MnistModel(nn.Module):
    def __init__(self, num_classes: int = 26) -> None:
        super().__init__()
        self.resnet = resnet101()
        self.classifier = \
            nn.Linear(1000, num_classes)

        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, x: Tensor) -> Tensor:
        x = self.resnet(x)
        x = self.classifier(x)

        return x

## 5. 학습

In [ ]:
def train(fold: int, verbose: int = 30) -> None:
    split_dataset('data/dirty_mnist_2nd_answer.csv')
    df = pd.read_csv('data/split_kfold.csv')
    df_train = df[df['kfold'] != fold].reset_index(drop=True)
    df_valid = df[df['kfold'] == fold].reset_index(drop=True)

    df_train.drop(['kfold'], axis=1).to_csv(f'data/train-kfold-{fold}.csv', index=False)
    df_valid.drop(['kfold'], axis=1).to_csv(f'data/valid-kfold-{fold}.csv', index=False)

    trainset = MnistDataset('data/train', f'data/train-kfold-{fold}.csv', transforms_train)
    train_loader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, num_workers=12)

    validset = MnistDataset('data/train', f'data/valid-kfold-{fold}.csv', transforms_test)
    valid_loader = DataLoader(validset, batch_size=128, shuffle=False, num_workers=12)

    num_epochs = 80
    device = 'cuda'
    scaler = GradScaler()

    model = NetMnistModel().to(device)
    model = nn.DataParallel(model, device_ids=[0, 1, 2, 3])

    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-5)
    criterion = nn.MultiLabelSoftMarginLoss()

    for epoch in range(num_epochs):
        model.train()
        for i, (images, targets) in enumerate(train_loader):
            optimizer.zero_grad()

            images = images.to(device)
            targets = targets.to(device)

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, targets)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if (i+1) % verbose == 0:
                outputs = outputs > 0.0
                acc = (outputs == targets).float().mean()
                print(f'Fold {fold} | Epoch {epoch} | L: {loss.item():.7f} | A: {acc.item():.7f}')

        if epoch > num_epochs-20 and epoch < num_epochs-1:
            model.eval()
            valid_acc = 0.0
            valid_loss = 0.0
            valid_size = valid_loader.batch_size
            for i, (images, targets) in enumerate(valid_loader):
                images = images.to(device)
                targets = targets.to(device)

                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, targets)

                valid_loss += loss.item()
                outputs = outputs > 0.0
                valid_acc += (outputs == targets).float().mean()

            print(f'Fold {fold} | Epoch {epoch} | L: {valid_loss/valid_size:.7f} | A: {valid_acc/valid_size:.7f}\n')

        if epoch > num_epochs-20 and epoch < num_epochs-1:
            torch.save(model.state_dict(), f'resnet101-f{fold}-{epoch}.pth')

In [ ]:
if __name__ == '__main__':
    train(0)
    train(1)
    train(2)
    train(3)
    train(4)

## 6. 테스트셋 제출 

In [ ]:
def load_model(fold: int, epoch: int, device: torch.device = 'cuda') -> nn.Module:
    model = MnistModel().to(device)
    state_dict = {}
    for k, v in torch.load(f'resnet-f{fold}-{epoch}.pth').items():
        state_dict[k[7:]] = v

    model.load_state_dict(state_dict)

    return model

In [ ]:
def test(device: torch.device = 'cuda'):
    submit = pd.read_csv('data/sample_submission.csv')

    model1 = load_model(0, 50)
    model2 = load_model(1, 50)
    model3 = load_model(2, 50)
    model4 = load_model(3, 50)
    model5 = load_model(4, 50)

    model1 = nn.DataParallel(model1, device_ids=[0, 1, 2, 3])
    model2 = nn.DataParallel(model2, device_ids=[0, 1, 2, 3])
    model3 = nn.DataParallel(model3, device_ids=[0, 1, 2, 3])
    model4 = nn.DataParallel(model4, device_ids=[0, 1, 2, 3])
    model5 = nn.DataParallel(model5, device_ids=[0, 1, 2, 3])

    model1.eval()
    model2.eval()
    model3.eval()
    model4.eval()
    model5.eval()

    batch_size = test_loader.batch_size
    batch_index = 0
    for i, (images, targets) in enumerate(test_loader):
        images = images.to(device)
        targets = targets.to(device)

        outputs1 = model1(images)
        outputs2 = model2(images)
        outputs3 = model3(images)
        outputs4 = model4(images)
        outputs5 = model5(images)

        outputs = (outputs1 + outputs2 + outputs3 + outputs4 + outputs5) / 5

        outputs = outputs > 0.0
        batch_index = i * batch_size
        submit.iloc[batch_index:batch_index+batch_size, 1:] = \
            outputs.long().squeeze(0).detach().cpu().numpy()

    submit.to_csv('resnet101-e50-kfold.csv', index=False)


if __name__ == '__main__':
    test()